### 4.Generative Adversarial Network / 적대적 생성 네트워크

- 인공 데이터 재생산 방법을 학습
    - WaveNet : 사람 목소리와 악기 소리 재생산
    - Zhang, Han, et al. "Stackgan: Text to photo-realistic image synthesis with stacked generative adversarial networks." Proceedings of the IEEE International Conference on Computer Vision. 2017. 텍스트를 기반으로 이미지 생성
    
    
- 두 개의 신경망을 동시에 학습한다.
    - 생성기 : 위조
    - 판별기 : 얼마나 진짜 같은지 판단
    
    

Keras-adversarial : 파이썬 패키지

git clone --depth=50 --branch=master https://github.com/bstriner/keras-adversarial.git

In [ ]:
import matplotlib as mpl

mpl.use('Agg')

In [ ]:
from keras.layers import Flatten, Dropout, LeakyReLU, Input, Activation
from keras.models import Model
from keras.layers.convolutional import UpSampling2D
from keras.optimizers import Adam
from keras.datasets import mnist

import pandas as pd
import numpy as np

In [ ]:
from keras_adversarial.legacy import Dense, BatchNormalization, Convolution2D
from keras_adversarial.image_grid_callback import ImageGridCallback
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling
from image_utils import dim_ordering_fix, dim_ordering_input, dim_ordering_reshape, dim_ordering_unfix

In [ ]:
# n targets k players

def gan_targets(n):
    generator_fake = np.ones((n, 1))
    generator_real = np.zeros((n, 1))
    discriminator_fake = np.zeros((n, 1))
    discriminator_real = np.ones((n, 1))
    
    return [generator_fake, generator_real, discriminator_fake, discriminator_real]

In [ ]:
def model_generator():
    nch = 256
    g_input = Input(shape=[100])
    H = Dense(nch * 14 * 14)(g_input)
    H = BatchNormalization(mode=2)(H)
    H = Activation('relu')(H)
    H = dim_ordering_reshape(nch, 14)(H)
    H = UpSampling2D(size=(2,2))(H)
    H = Convolution2D(int(nch /2), 3, 3, border_mode="same")(H)
    H = BatchNormalization(mode=2, axis=1)(H)
    H = Activation('relu')(H)
    H = Convolution2D(int(nch /4), 3, 3, border_mode="same")(H)
    H = BatchNormalization(mode=2, axis=1)(H)
    H = Activation('relu')(H)
    H = Convolution2D(1, 1, 1, border_mode="same")(H)
    g_V = Activation('sigmoid')(H)
    return Model(g_input, g_V)

In [ ]:
def model_discriminator(input_shape=(1,28,28), dropout_rate=0.5):
    d_input = dim_ordering_input(input_shape, name="input_x")
    nch = 512
    H = Convolution2D(int(nch /2), 5, 5, subsample=(2,2), border_mode="same", activation='relu')(d_input)
    H = LeakyReLU(0.2)(H)
    H = Dropout(dropout_rate)(H)
    H = Convolution2D(nch, 5, 5, subsample=(2,2), border_mode="same", activation='relu')(H)
    H = LeakyReLU(0.2)(H)
    H = Dropout(dropout_rate)(H)
    H = Flatten()(H)
    H = Dense(int(nch/2))(H)
    H = LeakyReLU(0.2)(H)    
    H = Dropout(dropout_rate)(H)
    d_V = Dense(1, activation='sigmoid')(H)
    return Model(d_input, d_V)

In [ ]:
def mnist_process(x):
    x = x.astype(np.float32)/255.0
    return x

def mnist_data():
    (xtrain, ytrain), (xtest,ytest) = mnist.load_data()
    return mnist_process(xtrain), mnist_process(xtest)

In [ ]:
latent_dim = 100

input_shape= (1, 28, 28)
generator = model_generator()
discriminator = model_discriminator(input_shape=input_shape)

gan = simple_gan(generator, discriminator, normal_latent_sampling((latent_dim,)))

generator.summary()
discriminator.summary()

gan.summary()

In [ ]:
# GAN

model = AdversarialModel(base_model=gan,\
                        player_params =[generator.trainable_weights, \
                                       discriminator.trainable_weights],\
                        player_names=['generator', 'discriminator'])


In [ ]:
model.adversarial_compile(adversarial_optimizer=AdversarialOptimizerSimultaneous(),\
                         player_optimizers=[Adam(1e-4, decay=1e-4), Adam(1e-3, decay=1e-4)],\
                         loss='binary_crossentropy')

In [ ]:
def generator_sampler():
    zsamples = np.random.normal(size=(10*10, latent_dim))
    gen = dim_ordering_unfix(generator.predict(zsamples))
    return gen.reshape((10, 10, 28, 28))

In [ ]:
generator_cb = ImageGridCallback("gan_output/gan_conv/epoch-{:03d}.png", generator_sampler)

xtrain, xtest = mnist_data()
xtrain = dim_ordering_fix(xtrain.reshape((-1, 1, 28, 28)))
xtest = dim_ordering_fix(xtest.reshape((-1, 1, 28, 28)))

y = gan_targets(xtrain.shape[0])
ytest = gan_targets(xtest.shape[0])

In [ ]:
history = model.fit(x=xtrain, y=y,\
                   validation_data=(xtest, ytest), \
                    callbacks=[generator_cb], nb_epoch=100, batch_size=32)
df = pd.DataFrame(history.history)

In [ ]:
df.to_csv("gan_output/gan_conv/history.csv")
generator.save("gan_output/gan_conv/generator.h5")
discriminator.save("gan_output/gan_conv/discriminator.h5")